#  BOE api

#### Import dependencies

In [1]:
# %pip install os json requests datetime

In [15]:
# tested in Python >= 3.12, < 3.13, use main pyproject.toml
import os
import json
import requests
from datetime import timedelta, datetime, date

## Download the BOE_sumario

In [8]:
def download_boe_sumario(fecha: str) -> bool:
    """
    Descarga el sumario del BOE para una fecha específica y lo guarda en formato JSON.
    """
    # Obtener el directorio actual
    current_dir = os.getcwd() # Directorio actual de ejecución del script

    #DIrectorio de gurdado de json de los boe
    boe_dir = os.path.join(current_dir, f"../data/output/boe/sumario/{fecha[:4]}") # Directorio de guardado de los JSON del BOE en subdirectorio por año

    # Crar el directorio si no existe
    os.makedirs(boe_dir, exist_ok=True)
    print(f"Directorio de guardado: {boe_dir}")

    # Configurar la solicitud
    url = f"https://boe.es/datosabiertos/api/boe/sumario/{fecha}" # Formato de fecha : AAAAmmdd (20240101)
    headers = {"Accept": "application/json"}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Guardar el JSON
        filename = f"{boe_dir}/BOE_sumario_{fecha}.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(response.json(), f, indent=2, ensure_ascii=False)

        print(f"Sumario de {fecha} guardado en {filename}")

    except requests.exceptions.HTTPError as e:
        print(f"Error {e.response.status_code} para fecha {fecha}")
    except requests.exceptions.Timeout:
        print(f"Timeout para la solicitud a {url}")
    except requests.exceptions.RequestException as e:
        print(f"Error en la solicitud: {e}")
    except json.JSONDecodeError:
        print(f"Error al decodificar la respuesta JSON para {fecha}")
    except OSError as e:
        print(f"Error al guardar el archivo: {e}")
    except Exception as e:
        print(f"Error inesperado: {e}")

    return True

### Sample of getting a BOE_sumario

From 1/1/2025 to 10/1/2025

In [14]:
start_date = date(2025, 1, 1)
end_date = date(2025, 1, 10)
delta = timedelta(days=1)

current = start_date
while current <= end_date:
    FECHA = current.strftime("%Y%m%d")
    download_boe_sumario(FECHA)
    current += delta

Directorio de guardado: /home/xgen0/CertyChain/CertyLex/backend/samples/notebooks/../data/output/boe/sumario/2025
Sumario de 20250101 guardado en /home/xgen0/CertyChain/CertyLex/backend/samples/notebooks/../data/output/boe/sumario/2025/BOE_sumario_20250101.json
Directorio de guardado: /home/xgen0/CertyChain/CertyLex/backend/samples/notebooks/../data/output/boe/sumario/2025
Sumario de 20250102 guardado en /home/xgen0/CertyChain/CertyLex/backend/samples/notebooks/../data/output/boe/sumario/2025/BOE_sumario_20250102.json
Directorio de guardado: /home/xgen0/CertyChain/CertyLex/backend/samples/notebooks/../data/output/boe/sumario/2025
Sumario de 20250103 guardado en /home/xgen0/CertyChain/CertyLex/backend/samples/notebooks/../data/output/boe/sumario/2025/BOE_sumario_20250103.json
Directorio de guardado: /home/xgen0/CertyChain/CertyLex/backend/samples/notebooks/../data/output/boe/sumario/2025
Sumario de 20250104 guardado en /home/xgen0/CertyChain/CertyLex/backend/samples/notebooks/../data/ou

### Auxiliar Functions

In [12]:
# Función auxiliar para descargar un documento dado su URL y extensión
def download_documento(url: str , ext: str, output_dir: str, identificador: str) -> None:
    """
    Descarga un documento desde una URL y lo guarda en el directorio especificado.
    Parámetros:
        url (str): URL del documento a descargar.
        ext (str): Extensión del documento (html, xml, pdf).
        output_dir (str): Directorio donde se guardará el documento.
        identificador (str): Identificador del documento.
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        # Se guarda el contenido en modo binario
        filename = os.path.join(output_dir, f"{identificador}.{ext}")
        with open(filename, "wb") as f: # Binary write (wb) does not require encoding
            f.write(response.content)
        print(f"Descargado {ext.upper()} para {identificador}: {filename}")
    except requests.exceptions.HTTPError as e:
        print(f"Error HTTP {e.response.status_code} al descargar {url}")
    except requests.exceptions.Timeout:
        print(f"Timeout al descargar {url}")
    except requests.exceptions.RequestException as e:
        print(f"Error en la solicitud de {url}: {e}")

In [ ]:
def download_boe_documentos(json_filepath: str):
    """
    Lee el archivo JSON del sumario del BOE, extrae los documentos referenciados 
    y los descarga en la carpeta de salida organizada por año y fecha.
    
    Los documentos se descargan en formato HTML, XML y PDF (si están disponibles).
    
    Parámetros:
      json_filepath (str): Ruta completa al archivo JSON del sumario.
      output_base_dir (str): Carpeta base donde se guardarán los documentos descargados.
    """
    # Fecha de publicación (por ejemplo, "20150101")
    fecha_publicacion = json_filepath.rsplit("_", maxsplit=-1)[-1].replace(".json", "")

    # ./backend/data/boe/diario
    current_os = os.getcwd() # Directorio actual de ejecución del script
    output_base_dir = os.path.join(current_os, "../data/output/boe/diario") # Directorio de guardado de los JSON del BOE en subdirectorio por año
    # Crear el directorio de salida: output_base_dir/{year}/{fecha_publicacion}/
    output_dir = os.path.join(output_base_dir, fecha_publicacion[:4], fecha_publicacion)
    os.makedirs(output_dir, exist_ok=True)
    print(f"Creando directorio de salida: {output_dir}")

    # Leer el JSON
    try:
        with open(json_filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
    except Exception as e:
        print(f"Error al leer {json_filepath}: {e}")
        return

    # Obtener la fecha de publicación (por ejemplo, "20150101") del JSON
    fecha_publicacion = data.get("data", {}).get("sumario", {}).get("metadatos", {}).get("fecha_publicacion")
    if not fecha_publicacion:
        print(f"No se encontró 'fecha_publicacion' en {json_filepath}")
        return

    # Recorrer la estructura del sumario para extraer las URL de cada documento
    diarios = data.get("data", {}).get("sumario", {}).get("diario", [])
    if not isinstance(diarios, list):
        diarios = [diarios]

    for diario in diarios:
        secciones = diario.get("seccion", [])
        if not isinstance(secciones, list):
            secciones = [secciones]
        for seccion in secciones:
            departamentos = seccion.get("departamento")
            if departamentos:
                if not isinstance(departamentos, list):
                    departamentos = [departamentos]
                for dept in departamentos:
                    # Los epígrafes pueden estar anidados en 'texto' o directamente en 'epigrafe'
                    if "texto" in dept:
                        epigrafes = dept["texto"].get("epigrafe", [])
                    else:
                        epigrafes = dept.get("epigrafe", [])
                    if not isinstance(epigrafes, list):
                        epigrafes = [epigrafes]
                    for epigrafe in epigrafes:
                        items = epigrafe.get("item")
                        if not items:
                            continue
                        if not isinstance(items, list):
                            items = [items]
                        for item in items:
                            identificador = item.get("identificador", "sin_id")
                            url_xml = item.get("url_xml")
                            if url_xml:
                                xml_output_dir = os.path.join(output_dir, "xml")
                                os.makedirs(xml_output_dir, exist_ok=True)
                                download_documento(url_xml, "xml", xml_output_dir, identificador)
                            else:
                                print(f"No se encontró URL XML ni HTML para {identificador}")

## Get all the XML_diario of each BOE_sumario

In [20]:
def download_rango_fechas(start_date: str, end_date: str, base_path: str):
    """
    Procesa todos los archivos JSON en un rango de fechas y ejecuta la función download_boe_documentos.
    
    Parámetros:
        start_date (str): Fecha de inicio en formato YYYYMMDD.
        end_date (str): Fecha de fin en formato YYYYMMDD.
        base_path (str): Ruta base donde se encuentran los archivos JSON.
    """
    current_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    while current_date <= end_date:
        fecha_str = current_date.strftime("%Y%m%d")
        json_filepath = os.path.join(base_path, fecha_str[:4], f"BOE_sumario_{fecha_str}.json")

        if os.path.exists(json_filepath):
            print(f"Procesando archivo: {json_filepath}")
            download_boe_documentos(json_filepath)
        else:
            print(f"Archivo no encontrado: {json_filepath}")

        current_date += timedelta(days=1)

In [ ]:
current_dir = os.getcwd() # Directorio actual de ejecución del script
BASE_PATH = os.path.join(current_dir, "../data/output/boe/sumario/" )
# Rango de fechas
START_DATE = "20250101"
END_DATE = "20250101"

download_rango_fechas(START_DATE, END_DATE, BASE_PATH)

Procesando archivo: c:\Users\manue\CertyChain\CertyLex\backend\sample\notebooks\../data/boe/sumario/2025\BOE_sumario_20250101.json
Creando directorio de salida: c:\Users\manue\CertyChain\CertyLex\backend\sample\notebooks\../data/boe/diario\2025\20250101
Descargado XML para BOE-A-2025-1: c:\Users\manue\CertyChain\CertyLex\backend\sample\notebooks\../data/boe/diario\2025\20250101\xml\BOE-A-2025-1.xml
Descargado XML para BOE-A-2025-2: c:\Users\manue\CertyChain\CertyLex\backend\sample\notebooks\../data/boe/diario\2025\20250101\xml\BOE-A-2025-2.xml
Descargado XML para BOE-A-2025-3: c:\Users\manue\CertyChain\CertyLex\backend\sample\notebooks\../data/boe/diario\2025\20250101\xml\BOE-A-2025-3.xml
Descargado XML para BOE-A-2025-4: c:\Users\manue\CertyChain\CertyLex\backend\sample\notebooks\../data/boe/diario\2025\20250101\xml\BOE-A-2025-4.xml
Descargado XML para BOE-A-2025-5: c:\Users\manue\CertyChain\CertyLex\backend\sample\notebooks\../data/boe/diario\2025\20250101\xml\BOE-A-2025-5.xml
Descar